In [ ]:
from jax import jit, random
import pandas as pd
from datetime import datetime, timedelta
import numpyro
from numpyro import distributions as dist
from numpyro import infer
import arviz as az
from IPython.display import Markdown
from plotly.express.colors import qualitative as qual_colours
import pathlib
import math
import plotly.express as px
import plotly.graph_objects as go

from estival.sampling import tools as esamp

from emu_renewal.process import CosineMultiCurve
from emu_renewal.distributions import GammaDens
from emu_renewal.renew import RenewalModel
from emu_renewal.outputs import get_spaghetti_from_params, get_quant_df_from_spaghetti, plot_spaghetti
from emu_renewal.outputs import plot_uncertainty_patches, PANEL_SUBTITLES, plot_3d_spaghetti, plot_post_prior_comparison
from emu_renewal.calibration import StandardCalib
from emu_renewal.utils import get_adjust_idata_index, adjust_summary_cols

In [ ]:
# Set path to target data from WHO weekly time series
target_data_path = pathlib.Path('../data/target-data/case-data.csv')

In [ ]:
# Load in target data from WHO weekly time series
data = pd.read_csv(target_data_path, index_col=0)
data.index = pd.to_datetime(data.index)

In [ ]:
# Also load daily time series data from owid
mys_data_daily = pd.read_csv("https://github.com/monash-emu/wpro_working/raw/main/data/new_cases.csv", index_col=0)["MYS"]
mys_data_daily.index = pd.to_datetime(mys_data_daily.index)

In [ ]:
mys_data_daily.plot()

In [ ]:
data['New_cases_MYS'].plot()

In [ ]:
data

In [ ]:
# Specify fixed parameters and get calibration data
proc_update_freq = 14
init_time = 50
mys_data = data['New_cases_MYS']
pop = 33e6
analysis_start = datetime(2021, 3, 1)
analysis_end = datetime(2021, 11, 1)
init_start = analysis_start - timedelta(init_time)
init_end = analysis_start - timedelta(1)
select_data = mys_data.loc[analysis_start: analysis_end]
init_data = mys_data.loc[init_start: init_end]

In [ ]:
proc_fitter = CosineMultiCurve()
renew_model = RenewalModel(33e6, analysis_start, analysis_end, proc_update_freq, proc_fitter, GammaDens(), 50, init_data, GammaDens())

In [ ]:
# Define parameter ranges
priors = {
    "gen_mean": dist.TruncatedNormal(5.0, 0.4, low=0.0),
    "gen_sd": dist.TruncatedNormal(3.8, 0.5, low=0.0),
    "cdr": dist.Beta(4.0, 10.0),
    "rt_init": dist.Normal(0.0, 0.25),
    "report_mean": dist.TruncatedNormal(5, 0.5, low=0.0),
    "report_sd": dist.TruncatedNormal(2, 0.5, low=0.0),
}

In [ ]:
calib = StandardCalib(renew_model, priors, select_data, indicator='weekly_sum')
kernel = infer.NUTS(calib.calibration, dense_mass=True, init_strategy=infer.init_to_uniform(radius=0.5))
mcmc = infer.MCMC(kernel, num_chains=2, num_samples=1000, num_warmup=1000)
mcmc.run(random.PRNGKey(1))

In [ ]:
idata = az.from_dict(mcmc.get_samples(True))
idata_sampled = az.extract(idata, num_samples=200)
sample_params = esamp.xarray_to_sampleiterator(idata_sampled)

In [ ]:
def get_full_result(gen_mean, gen_sd, proc, cdr, rt_init, report_mean, report_sd):
    return renew_model.renewal_func(gen_mean, gen_sd, proc, cdr, rt_init, report_mean, report_sd)

full_wrap = jit(get_full_result)
spaghetti = get_spaghetti_from_params(renew_model, sample_params, full_wrap)
quantiles_df = get_quant_df_from_spaghetti(renew_model, spaghetti, quantiles=[0.05, 0.5, 0.95])
plot_uncertainty_patches(quantiles_df, select_data, qual_colours.Plotly).update_layout(showlegend=False)

In [ ]:
az.summary(idata)

In [ ]:
plot_post_prior_comparison(idata, list(priors.keys()), priors);

In [ ]:
model_daily_cases = quantiles_df['cases'][0.50]

In [ ]:
model_daily_cases = pd.DataFrame(model_daily_cases)

In [ ]:
model_daily_cases['date'] = model_daily_cases.index()

In [ ]:
# create date column and reset index

model_daily_cases = model_daily_cases.reset_index()
model_daily_cases = model_daily_cases.rename(columns={"index": "date"})

In [ ]:
import numpy as np

In [ ]:
# this gives the monday of each week
model_daily_cases['week_start'] = model_daily_cases['date'] + (np.timedelta64(6, 'D') - model_daily_cases['date'].dt.weekday.astype('timedelta64[D]')) 

In [ ]:
model_daily_cases['date'].dt.weekday.astype('timedelta64[D]')

In [ ]:
weekly_cases = model_daily_cases.groupby(['week_start'])[0.5].sum()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=weekly_cases.index, y=weekly_cases,  name='model'))
              
fig.add_trace(go.Scatter(x=data.index, y=data['New_cases_MYS'], name='WHO data'))

fig.update_xaxes(range=(datetime(2021, 3, 1), datetime(2021, 10, 18)), title='date')
fig.update_yaxes(title='weekly reported cases')

fig.show()